# Clasificación de señales de tráfico usando TensorFlow y Keras

Instalación de dependencias:

In [ ]:
!pip install tensorflow==2.10 opencv-python matplotlib

In [ ]:
!pip list

In [3]:
import tensorflow as t
import os

import cv2
import imghdr
import numpy as np
from matplotlib import pyplot as plt

In [4]:
gpus = t.config.experimental.list_physical_devices('GPU')
len(gpus) # 1 -> GPU disponible

1

Funciones sobre imágenes:

In [7]:
data_dir = 'data'

# Mostrar una imagen con matplotlib usando cv2 para corregir los colores
def mostrarImagen(img: cv2.typing.MatLike):
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))

In [ ]:
# Ejemplo de lectura de imagen
img = cv2.imread(os.path.join(data_dir, 'stop-sign', '1c56eb1de271aaca19a8f081dca6e5eb.jpg'))
mostrarImagen(img)

Cargar las imágenes:

In [17]:
datos = t.keras.utils.image_dataset_from_directory(data_dir)

Found 423 files belonging to 4 classes.


In [18]:
# Creamos un iterador para recorrer los datos
iterador_datos = datos.as_numpy_iterator()

In [20]:
lote_datos = iterador_datos.next()

InvalidArgumentError: {{function_node __wrapped__IteratorGetNext_output_types_2_device_/job:localhost/replica:0/task:0/device:CPU:0}} Unknown image file format. One of JPEG, PNG, GIF, BMP required.
	 [[{{node decode_image/DecodeImage}}]] [Op:IteratorGetNext]

In [ ]:
lote_datos